# Compute the pairwise gene similarity

## Importing the libraries 

In [1]:
import os
import pandas as pd
import numpy as np
import pickle

import mygene
from goatools.base import get_godag
from goatools.semsim.termwise.wang import SsWang # GO terms similarity defined in the Wang paper

## Importing the dataset

In [2]:
data_folder_path = os.path.join(os.getcwd(),'data')
data_path = os.path.join(data_folder_path, 'DG-Miner_miner-disease-gene.tsv')

data = pd.read_csv(data_path, sep='\t', header=0, names=['diseases','genes'])

In [ ]:
print("Number of links: ", len(data.index))
print('\n')
print("Unique values:")
print(data.nunique())

Number of links:  15509619


Unique values:
diseases     5664
genes       17822
dtype: int64


## Getting the GO terms associated to each gene in our dataset using the *mygene* library

In [ ]:
missing_genes_count = 0
MF_GO_terms_dict = {}
mg = mygene.MyGeneInfo()
i = 0

for gene_code in data['genes'].unique():
    
    query_res = mg.query(gene_code, size=1)
    if len(query_res['hits']) == 0:
        missing_genes_count += 1
        continue
    
    gene_id = query_res['hits'][0]['_id']
    gene_dict = mg.getgene(gene_id)
    if 'go' not in gene_dict:
        missing_genes_count += 1
        continue
    
    gene_GO_terms_dict = gene_dict['go']
    if 'MF' not in gene_GO_terms_dict:
        missing_genes_count += 1
        continue
    
    gene_MF_GO_terms_dicts = gene_GO_terms_dict['MF']
    gene_MF_GO_terms = set()
    if type(gene_MF_GO_terms_dicts) != list:
        gene_MF_GO_terms.add(gene_MF_GO_terms_dicts['id'][3:])
    else:
        for MF_GO_term_dict in gene_MF_GO_terms_dicts:
            gene_MF_GO_terms.add(MF_GO_term_dict['id'][3:])
    MF_GO_terms_dict[gene_code] = gene_MF_GO_terms

    i += 1
    if i%1000 == 0:
        print("Processed ", i, " genes")

Processed  1  genes
Processed  1001  genes
Processed  2001  genes
Processed  3001  genes
Processed  4001  genes
Processed  5001  genes
Processed  6001  genes
Processed  7001  genes
Processed  8001  genes
Processed  9001  genes
Processed  10001  genes
Processed  11001  genes
Processed  12001  genes
Processed  13001  genes
Processed  14001  genes
Processed  15001  genes
Processed  16001  genes


In [ ]:
print(missing_genes_count)

1170


In [1]:
# Uncomment this code if you don't have the file containing the dictionary (key, value) ---> (gene_code, GO_terms), that is the GO_terms_dict.txt file present in the repo.
'''
with open('GO_terms_dict.txt', 'wb') as handle:
    pickle.dump(MF_GO_terms_dict, handle)
'''


"\nwith open('GO_terms_dict.txt', 'wb') as handle:\n    pickle.dump(MF_GO_terms_dict, handle)\n"

In [2]:
with open('GO_terms_dict.txt', 'rb') as handle:
    MF_GO_terms_dict = pickle.loads(handle.read())

In [ ]:
print(len(MF_GO_terms_dict))

16652


## Retrieving GO terms ontology from the web

In [3]:
godag_path = os.path.join(data_folder_path, 'go-basic.obo')
godag = get_godag(godag_path)

  EXISTS: ./data/go-basic.obo
./data/go-basic.obo: fmt(1.2) rel(2021-01-01) 47,285 GO Terms


In [4]:
for gene_code in MF_GO_terms_dict:
    MF_GO_terms_dict[gene_code] = set(map(lambda term: 'GO:' + term, MF_GO_terms_dict[gene_code])) # We add the prefix "GO:" to each term in our dict because it is needed for the goatools library

In [5]:
goids = set()
for terms in MF_GO_terms_dict.values():
    goids = goids.union(terms)
print("Number of different GO terms in our dataset:", len(goids))

Number of different GO terms in our dataset: 4285


In [6]:
wang = SsWang(goids, godag) # We pass the all the GO terms between which we have to compute the pairwise GO term similarity and we see that some GO terms are not present in the godag ---> we remove all associated genes, because for these we cannot compute the genes simialrity

**WARNING: GO IDs NOT FOUND: ['GO:0030375', 'GO:0045155']


In [7]:
genes_to_delete = []

for gene_code in MF_GO_terms_dict:
    if 'GO:0030375' in MF_GO_terms_dict[gene_code] or 'GO:0045155' in MF_GO_terms_dict[gene_code]:
        genes_to_delete.append(gene_code)

In [8]:
genes_to_delete

['Q15648', 'P99999', 'F6THM6', 'Q9Y2X0', 'P08574', 'Q15596']

In [9]:
for gene_to_delete in genes_to_delete:
    MF_GO_terms_dict.pop(gene_to_delete)

In [10]:
goids.discard('GO:0030375')
goids.discard('GO:0045155')
print("Number of different GO terms in our dataset after the removal of non-present terms in the DAG:", len(goids))

Number of different GO terms in our dataset after the removal of non-present terms in the DAG: 4283


In [ ]:
wang = SsWang(goids, godag)

## Computing pairwise gene similarity

In [ ]:
def term_set_similarity(GO_term, GO_terms):
    max_sim = 0.0
    for term in GO_terms:
        sim = wang.get_sim(GO_term, term)
        if sim > max_sim:
            max_sim = sim
    return max_sim

def genes_similarity(GO_terms1, GO_terms2):
    similarities_sum = 0.0
    for GO_term in GO_terms1:
        similarities_sum += term_set_similarity(GO_term, GO_terms2)
    for GO_term in GO_terms2:
        similarities_sum += term_set_similarity(GO_term, GO_terms1)
    return similarities_sum / (len(GO_terms1) + len(GO_terms2))


genes_GO_terms = list(MF_GO_terms_dict.values())
num_genes = len(genes_GO_terms)
genes_features_matrix = np.zeros( (num_genes, num_genes) )
for i in range(num_genes - 1):
    terms1 = genes_GO_terms[i]
    for j in range(i + 1, num_genes):
        sim = genes_similarity(terms1, genes_GO_terms[j])
        genes_features_matrix[i][j] = genes_features_matrix[j][i] = sim

In [ ]:
np.fill_diagonal(genes_features_matrix, 1.0)

In [ ]:
np.count_nonzero(genes_features_matrix==0.0)

0

In [ ]:
np.amin(genes_features_matrix)

0.019664435298605537

In [ ]:
np.save('genes_features_matrix.npy', genes_features_matrix)